## Scratch Book to work out overview image

## Fix the YTD stats by putting the correct roster into it

In [6]:
import pandas as pd
import numpy as np
import sqlite3
import os

roster_path = '../../data/rosters/all_time_combined_roster.csv'
# Read as dataframe
roster = pd.read_csv(roster_path)
# roster.head()

db_path = '../../data/db/CHN_YTD_Stats.db'
conn = sqlite3.connect(db_path)
c = conn.cursor()

# Get all tables in the database
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = c.fetchall()
# print tables in the database
for table in tables:
    print(table)



('goalie_stats',)
('line_chart',)
('advanced_metrics',)
('game_details',)
('player_stats',)
('linescore',)
('penalty_summary',)
('scoring_summary',)
('player_stats_ytd',)
('master_roster',)


C:\Users\jbanc\AppData\Local\Temp\ipykernel_12472\3269926165.py:8: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  roster = pd.read_csv(roster_path)


In [11]:
# Filter the DataFrame for the 2023 season
filtered_roster_2023_df = roster[roster['Season'] == 2023]

# Show the size of the filtered DataFrame to confirm it looks reasonable
filtered_roster_2023_df_size = filtered_roster_2023_df.shape

filtered_roster_2023_df_size
filtered_roster_2023_df.head()


,Team,Clean_Name,No,Position,Yr,Ht,Wt,DOB,Hometown,Height_Inches,Draft_Year,NHL_Team,D_Round,Last Team,League,Season,file,captain
76800,Lake Superior,Jack Blanchett,16,Defensemen,Fr,11-May,185,5/12/2003,"Monroe, Mich.",71,0,NaN,0,Powell,BCHL,2023,roster_data_2023.csv,NaN
76801,Lake Superior,Artyom Borshyov,23,Defensemen,Sr,3-Jun,210,8/22/2000,"Vitebsk, Belarus",75,0,NaN,0,Youngstown,USHL,2023,roster_data_2023.csv,NaN
76802,Lake Superior,Evan Bushy,5,Defensemen,Fr,1-Jun,195,3/26/2002,"Mankato, Minn.",73,0,NaN,0,Trail,BCHL,2023,roster_data_2023.csv,NaN
76803,Lake Superior,Jacob Conrad,4,Defensemen,Fr,11-May,180,5/18/2002,"Green Bay, Wis.",71,0,NaN,0,Fairbanks,NAHL,2023,roster_data_2023.csv,NaN
76804,Lake Superior,Jeremy Gervais,25,Defensemen,Sr,11-May,180,1/9/1999,"Prince George, B.C.",71,0,NaN,0,Salmon Arm,BCHL,2023,roster_data_2023.csv,NaN


In [8]:
# Remove the existing master_roster table
remove_table_query = "DROP TABLE IF EXISTS master_roster;"
conn.execute(remove_table_query)

# Commit the changes to the database
conn.commit()

# Verify the table has been removed by checking the list of tables again
tables_after_removal = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

tables_after_removal


,name
0,goalie_stats
1,line_chart
2,advanced_metrics
3,game_details
4,player_stats
5,linescore
6,penalty_summary
7,scoring_summary
8,player_stats_ytd


In [12]:
# Add the filtered 2023 roster back into the database as 'master_roster'
filtered_roster_2023_df.to_sql('master_roster', conn, if_exists='replace', index=False)

# Commit the changes to the database to ensure they are saved
conn.commit()

# Verify the table has been added by checking the list of tables again
tables_after_addition = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

# Display the updated list of tables to confirm the addition
print(tables_after_addition)

# Optionally, you can also inspect the first few rows of the new 'master_roster' table to confirm it's correctly populated
master_roster_preview = pd.read_sql("SELECT * FROM master_roster LIMIT 5;", conn)
print(master_roster_preview)


               name
0      goalie_stats
1        line_chart
2  advanced_metrics
3      game_details
4      player_stats
5         linescore
6   penalty_summary
7   scoring_summary
8  player_stats_ytd
9     master_roster
            Team       Clean_Name  No    Position  Yr      Ht   Wt        DOB  \
0  Lake Superior   Jack Blanchett  16  Defensemen  Fr  11-May  185  5/12/2003   
1  Lake Superior  Artyom Borshyov  23  Defensemen  Sr   3-Jun  210  8/22/2000   
2  Lake Superior       Evan Bushy   5  Defensemen  Fr   1-Jun  195  3/26/2002   
3  Lake Superior     Jacob Conrad   4  Defensemen  Fr  11-May  180  5/18/2002   
4  Lake Superior   Jeremy Gervais  25  Defensemen  Sr  11-May  180   1/9/1999   

              Hometown  Height_Inches  Draft_Year NHL_Team  D_Round  \
0        Monroe, Mich.             71           0     None        0   
1     Vitebsk, Belarus             75           0     None        0   
2       Mankato, Minn.             73           0     None        0   
3      Gr

# Work out Name parsing in trouble tables



In [9]:
### Book to collect data about the current college hockey season from College Hockey News

## Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import logging
from tqdm import tqdm
import re

from sqlalchemy import create_engine


import sqlite3

## global variables
current_year_url = 'https://www.collegehockeynews.com/schedules/?season=20232024'

tag = '2023_Season_10_31_23'


## Base usl for box scores and metrics
base_url = 'https://www.collegehockeynews.com'

## Functions
### Parse the current season schedule / results page

def parse_current_season(url):
        # Initialize variables
    current_date = None
    current_conference = None
    game_notes = None

    # Initialize an empty list to hold the data
    data = []

    # Parse the page with BeautifulSoup
    # Get the page with requests
    response = requests.get(url)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # select the table or tables
    tables = soup.find_all('table')

    rows = soup.find_all('tr')

    # Loop through each row to find relevant information
    for row in rows:
        # Check for date row
        if row.get('class') == ['stats-section']:
            current_date = row.find('td').text.strip()
        # Check for conference row
        elif row.get('class') == ['sked-header']:
            current_conference = row.find('td').text.strip()
        # Check for game notes
        elif len(row.find_all('td')) == 2:
            game_notes = row.find_all('td')[1].text.strip()
        # Process rows with game data
        elif row.get('valign') == 'top':
            cells = row.find_all('td')
            if len(cells) >= 9:
                home_team = cells[0].text.strip()
                home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
                home_score = cells[1].text.strip()
                away_team = cells[3].text.strip()
                away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
                away_score = cells[4].text.strip()
                ot = cells[5].text.strip()
                box_link = cells[7].find('a')['href'] if cells[7].find('a') else None
                metrics_link = cells[8].find('a')['href'] if cells[8].find('a') else None
                # Capture Game Notes
                game_notes_cell = cells[-1].find('small')
                game_notes = game_notes_cell.text.strip() if game_notes_cell else None

                # Append data to the list
                data.append([current_date, current_conference, game_notes, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, box_link, metrics_link])
                game_notes = None  # Reset game notes for the next row
    return data

## call the function
data = parse_current_season(current_year_url)


# Create a dataframe from the list

columns = ['Date', 'Conference', 'Game_Notes', 'Home_Team', 'Home_Team_Link', 'Home_Score', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'OT', 'Box_Link', 'Metrics_Link']
df = pd.DataFrame(data, columns=columns)
            
## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# remove day of the week from date
# format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

### Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Function to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    game_id = f'{row.Date}_{home_team_abbr}_{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}_{row.Home_Team}_{row.Away_Team}', axis=1)

## Filter out games that have not been played yet
df = df[df['Home_Score'] != '']

# Replace Nan values in metrics column with empty string
df['Metrics_Link'] = df['Metrics_Link'].fillna('')




In [10]:
# df.info()

    # Fetch HTML content for box score
box_score_response = requests.get(box_score_url)
box_score_html = box_score_response.text

NameError: name 'box_score_url' is not defined

In [ ]:


############# PARSEING SCORING SUMMARY WITH BS4

def parse_scoring_summary(html_content):
    # Initialize BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the scoring div and table
    scoring_div = soup.find('div', id='scoring')
    if scoring_div is None:
        logging.error("Scoring div not found")
        return None

    scoring_table = scoring_div.find('table')
    if scoring_table is None:
        logging.error("Scoring table not found within the scoring div")
        return None

    # Initialize list to store scoring events
    scoring_events = []
    period = None

    # Loop through table rows
    for row in scoring_table.find_all('tr'):
        if 'stats-section' in row.get('class', []):
            td = row.find('td')
            if td:
                period = td.text.strip()
            else:
                logging.warning("Period name not found in 'stats-section' row")
                period = "Unknown"
        else:
            cols = row.find_all('td')
            if len(cols) > 1:
                try:
                    team = cols[0].text.strip()
                    pp = cols[1].text.strip()

                    player_data = cols[3].text.strip()
                    match = re.match(r"(.+)\s\((\d+)\)", player_data)
                    player = match.group(1) if match else player_data
                    goals = int(match.group(2)) if match else None

                    assist_data_raw = cols[4].text.strip()
                    assist_data = assist_data_raw.split(", ") if assist_data_raw else []
                    assist1 = assist_data[0] if len(assist_data) > 0 else None
                    assist2 = assist_data[1] if len(assist_data) > 1 else None

                    time = cols[5].text.strip()

                    scoring_event = {
                        'Period': period,
                        'Team': team,
                        'PP': pp,
                        'Player': player,
                        'Player_Goals': goals,
                        'Assist1': assist1,
                        'Assist2': assist2,
                        'Time': time
                    }
                    scoring_events.append(scoring_event)
                except Exception as e:
                    logging.error(f"An error occurred while parsing a scoring event row: {e}")
            else:
                logging.warning(f"Insufficient columns in scoring row: {len(cols)}")

    return pd.DataFrame(scoring_events)

# Run the function
parse_scoring_summary(html_content)

# Break Different STUFF BELOW

In [ ]:
### COPY OF DEPENDENCIES AND SET UP FROM ADAPTABLE CODE BOOK

# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.legend_handler import HandlerTuple
from matplotlib.legend_handler import HandlerBase

import os


import seaborn as sns
import sqlite3

### Set the folder paths for the data
### DATA FOLDER PATHS
data_folder = '../data/' # ROSTER DATA & GAME STATS
## IMAGES FOLDER PATHS
logo_folder = '../images/logos/' 
background_folder = '../images/backgrounds/'

# Roster Data Paths
roster_path_current = 'rosters/2023_master_roster.csv'
roster_path_previous = 'rosters/2022_master_roster.csv'
# Once the Database Clean and Transform code is updated to insert the roster, we can use the data from the database

# Game Stats Database Path
db_folder = '../data/'
# db_path = '2023_YTD_Game_Stats_Cleaned.db'
db_path = '2023_Season_10_31_23_Game_Stats.db'

## Connect to the Database
conn = sqlite3.connect(db_folder + db_path)

## Load rosters into dataframes
df_recent = pd.read_csv(data_folder + roster_path_current)
df_last = pd.read_csv(data_folder + roster_path_previous)


## HOTFIX#####
# Replace dashes with spaces in the School column of the roster dataframes
df_recent['School'] = df_recent['School'].str.replace('-', ' ')
df_last['School'] = df_last['School'].str.replace('-', ' ')


In [ ]:
# TEST PARAMETERES

## Set paramaeters

### Set Matchup Tag - This will be used as a folder name to save the charts
matchup_tag = 'MSU_OSU_NEW' #

## Teams to Highlight - Use dashes instead of spaces
team_1 = 'Michigan State' # 
team_1_abrv = 'MSU' #
team_2 = 'Ohio State' #
team_2_abrv = 'OSU' #

## Path to School Logos
team_1_logo =  'msu.png' #
team_2_logo =  'osu.png' #
team_logos = {team_1: logo_folder + team_1_logo, team_2: logo_folder + team_2_logo}

## Set Colors
team_1_color =  'darkgreen' # For Michigan State
team_2_color =  'darkred' # For Ohio State
team_colors = {team_1: team_1_color, team_2: team_2_color}
############ Eventaully want to create a dictionary of colors for each team and save in a file to import

team_abrv_mapping = {
    'Michigan State': 'MSU',
    'Ohio State': 'OSU'
}


### Create a disctionary called highlight_colors to store the colors for each team
### Do it this way so i can reuse code from MSU_Explore_and_Plot notebook
highlight_colors = {team_1_abrv: team_1_color, team_2_abrv: team_2_color}

#### Set Background Image
path_to_bg_image =  'msu_osu_2.png' #
alpha_value = 0.1 # Set the transparency of the background image
## Load the background image
bg_img = mpimg.imread(background_folder + path_to_bg_image)



# Adjust the default chart size and aspect ratio
### SET THE FIGURE SIZE OF THE CHARTS ###
fig_deafult_1 = (10, 8) # Original
fig_deafult_2 = (12, 10)
# Turn on/off the figure size by commenting/uncommenting the lines below
figure_size = fig_deafult_1
# figure_size = fig_deafult_2


################################
## FONT PARAMETERES
font_title_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'bold', 'size': 36}
font_label_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'bold', 'size': 16}
font_tick_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'normal', 'size': 20}
font_label_large_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'bold', 'size': 20}
font_tick_large_param = {'family': 'sans-serif', 'color': 'darkslategray', 'weight': 'normal', 'size': 16}
#################################





## Create a scatter plot of the age and in days of all

In [ ]:
df_recent.head()

## Compare The Rosters

In [ ]:


## Function to compare the members of each team to the previous year and return information about roster turnover

from collections import Counter

def compare_rosters(df_recent, df_last, team):
    # Filter the dataframes by the team
    df_recent_team = df_recent[df_recent['School'] == team]
    df_last_team = df_last[df_last['School'] == team]
    
    # Set of player names for easier comparison
    recent_players = set(df_recent_team['Name'])
    last_players = set(df_last_team['Name'])
    
    # Calculate turnover stats
    new_players = recent_players - last_players
    departed_players = last_players - recent_players
    retained_players = recent_players.intersection(last_players)
    
    # Additional stats like turnover rate, etc.
    turnover_rate = len(departed_players) / len(last_players) * 100 if len(last_players) > 0 else 0
    
    # Count of players in each class rank for both data sets
    recent_class_count = Counter(df_recent_team['Yr.'])
    last_class_count = Counter(df_last_team['Yr.'])
    
    # Calculate how many players from each class were departed and retained
    departed_class_count = Counter(df_last_team[df_last_team['Name'].isin(departed_players)]['Yr.'])
    retained_class_count = Counter(df_last_team[df_last_team['Name'].isin(retained_players)]['Yr.'])
    
    return {
        'new_players': len(new_players),
        'departed_players': len(departed_players),
        'retained_players': len(retained_players),
        'turnover_rate': turnover_rate,
        'recent_class_count': recent_class_count,
        'last_class_count': last_class_count,
        'departed_class_count': departed_class_count,
        'retained_class_count': retained_class_count
    }


# Define a list of teams for analysis (you can replace this list with 'all_teams' or any specific list you have)
# teams_to_analyze = [team_1, team_2]

# Get unique team names from the roster data
all_teams = df_recent['School'].unique()
# Use the entire list of teams
teams_to_analyze = all_teams

# Initialize an empty list to store results
results_list = []

# Loop through each team and get the turnover stats
for team in teams_to_analyze:
    team_stats = compare_rosters(df_recent, df_last, team)
    team_stats['team'] = team  # Add team name to the result dictionary
    results_list.append(team_stats)

# Convert the list of dictionaries to a DataFrame
df_turnover_stats = pd.DataFrame(results_list)





df_turnover_stats.head()
# df_recent.head()

In [ ]:
df_recent.head()

## Merge in data on average team age and experience

In [ ]:
## Rename column on df_recent and _df_last to match the column on df_turnover_stats
df_recent = df_recent.rename(columns={'School': 'team'})
df_last = df_last.rename(columns={'School': 'team'})

# Calculate and store the avera age and experience level of each team
## Age - Use the DOB column to calculate each players age in data, average the age of each team, convert back to years (3 decimal places)
df_recent['Age'] = (pd.to_datetime('today') - pd.to_datetime(df_recent['DOB'])).dt.days
df_recent['Age'] = df_recent['Age'] / 365


df_last['Age'] = (pd.to_datetime('today') - pd.to_datetime(df_last['DOB'])).dt.days
df_last['Age'] = df_last['Age'] / 365


## Experience - Use the Yr. column to calculate each players experience in years, average the experience of each team, convert back to years (3 decimal places)
class_rank_mapping = {'Fr': 0, 'So': 1, 'Jr': 2, 'Sr': 3, 'Gr': 4}
df_recent['Experience'] = df_recent['Yr.'].map(class_rank_mapping)
df_last['Experience'] = df_last['Yr.'].map(class_rank_mapping)

# Caluclate the average age and experience of each team in each dataframe and store in a dictionary
team_age_dict_current = df_recent.groupby('team')['Age'].mean().round(3).to_dict()
team_exp_dict_current = df_recent.groupby('team')['Experience'].mean().round(3).to_dict()
team_age_dict_last = df_last.groupby('team')['Age'].mean().round(3).to_dict()
team_exp_dict_last = df_last.groupby('team')['Experience'].mean().round(3).to_dict()

## Add the average age and experience df_turnover_stats - Use the dictionaries created above, add current as age_current, add last as age_last
df_turnover_stats['age_current'] = df_turnover_stats['team'].map(team_age_dict_current)
df_turnover_stats['age_last'] = df_turnover_stats['team'].map(team_age_dict_last)

df_turnover_stats['exp_current'] = df_turnover_stats['team'].map(team_exp_dict_current)
df_turnover_stats['exp_last'] = df_turnover_stats['team'].map(team_exp_dict_last)


df_turnover_stats.head()

## Add Draft Information to DF

In [ ]:
df_recent.head()

## Merge Draft Data Into DataFrame

In [ ]:
# Create new column called "Drafted" and assign a value of 1 if the player was drafted, 0 if not
df_recent['Drafted'] = np.where(df_recent['Draft_Year'] > 0, 1, 0)
df_last['Drafted'] = np.where(df_last['Draft_Year'] > 0, 1, 0)



# Calculate the Draft Score Based on round drafted
### NEW DRAFT SCORE CALCULATION ###
# Assign weights to draft rounds
draft_weights = {
    1: 8,  # Extra weight for 1st round
    2: 6,
    3: 5,
    4: 4,
    5: 3,
    6: 2,
    7: 1
}

# Compute player draft scores and fill NaNs with 0
df_recent['Player_Draft_Score'] = df_recent['D_Round'].map(draft_weights).fillna(0)
df_last['Player_Draft_Score'] = df_last['D_Round'].map(draft_weights).fillna(0)

## Create new dataframes with team, count of drafted players, and sum of draft scores
df_recent_count = df_recent.groupby('team')['Drafted'].sum().reset_index()
df_recent_sum = df_recent.groupby('team')['Player_Draft_Score'].sum().reset_index()

df_last_count = df_last.groupby('team')['Drafted'].sum().reset_index()
df_last_sum = df_last.groupby('team')['Player_Draft_Score'].sum().reset_index()


# Merge the count and sum dataframes into single dataframe
df_recent_draft_score = df_recent_count.merge(df_recent_sum, on='team', how='left')
df_last_draft_score = df_last_count.merge(df_last_sum, on='team', how='left')

# Rename the columns
df_recent_draft_score = df_recent_draft_score.rename(columns={'Drafted': 'Drafted_Count', 'Player_Draft_Score': 'Player_Draft_Score_Sum'})
df_last_draft_score = df_last_draft_score.rename(columns={'Drafted': 'Drafted_Count_Previous', 'Player_Draft_Score': 'Player_Draft_Score_Sum_Previous'})

# Merge the two dataframes into one
df_draft_score = df_recent_draft_score.merge(df_last_draft_score, on='team', how='left')

# Normalize the two columns of draft scores to a 0-100 scale
df_draft_score['Normalized_Draft_Score'] = df_draft_score['Player_Draft_Score_Sum'] / df_draft_score['Player_Draft_Score_Sum'].max() * 100
df_draft_score['Normalized_Draft_Score_Previous'] = df_draft_score['Player_Draft_Score_Sum_Previous'] / df_draft_score['Player_Draft_Score_Sum_Previous'].max() * 100

# Calculate the change in draft score from last year
df_draft_score['Draft_Score_Change'] = df_draft_score['Normalized_Draft_Score'] - df_draft_score['Normalized_Draft_Score_Previous']
## Switch and store as percentage change
df_draft_score['Draft_Score_Change'] = df_draft_score['Draft_Score_Change'] / df_draft_score['Normalized_Draft_Score_Previous'] * 100

# Merge into the df_turnover_stats dataframe
df_turnover_stats = df_turnover_stats.merge(df_draft_score[['team', 'Drafted_Count', 'Drafted_Count_Previous', 'Draft_Score_Change', 'Normalized_Draft_Score', 'Normalized_Draft_Score_Previous']], on='team', how='left')

# Reorder columns so team is first
df_turnover_stats = df_turnover_stats[['team', 'new_players', 'departed_players', 'retained_players', 'turnover_rate', 'recent_class_count', 'last_class_count', 'departed_class_count', 'retained_class_count', 'age_current', 'age_last', 'exp_current', 'exp_last', 'Drafted_Count', 'Drafted_Count_Previous', 'Draft_Score_Change', 'Normalized_Draft_Score', 'Normalized_Draft_Score_Previous']]

# Calculate and store ranking for the Age and Experience columns
df_turnover_stats['age_rank_current'] = df_turnover_stats['age_current'].rank(ascending=False) # Oldest = 1 
df_turnover_stats['exp_rank_current'] = df_turnover_stats['exp_current'].rank(ascending=False) # Oldest = 1


## Save dataframe as csv in TEMP folder
df_turnover_stats.to_csv('../TEMP/df_turnover_stats.csv', index=False)

# df_turnover_stats.head(35)



## Calculate each teams current W/L/T Record from Database

In [ ]:
# Query to get the required columns for calculating win/lose/tie records
query_linescore_for_records = """
SELECT Team, goalsT, Game_ID FROM linescore;
"""

# Execute the query and convert to a DataFrame
linescore_records_df = pd.read_sql(query_linescore_for_records, conn)

# Initialize empty dictionary to store win/lose/tie records
team_records = {}

# Loop through the DataFrame grouped by Game_ID to find winner, loser, or tie for each game
for game_id, game_data in linescore_records_df.groupby("Game_ID"):
    team1, team2 = game_data["Team"].values
    goals_team1, goals_team2 = game_data["goalsT"].values
    
    # Initialize the team record if not already present
    for team in [team1, team2]:
        if team not in team_records:
            team_records[team] = {"Wins": 0, "Losses": 0, "Ties": 0}
    
    # Determine the outcome of the game for each team
    if goals_team1 > goals_team2:
        team_records[team1]["Wins"] += 1
        team_records[team2]["Losses"] += 1
    elif goals_team2 > goals_team1:
        team_records[team2]["Wins"] += 1
        team_records[team1]["Losses"] += 1
    else:
        team_records[team1]["Ties"] += 1
        team_records[team2]["Ties"] += 1

# Convert the records dictionary to a DataFrame for better display
records_df = pd.DataFrame.from_dict(team_records, orient="index").reset_index()
records_df.rename(columns={"index": "Team"}, inplace=True)

records_df.tail()  # Display the first few rows
# len(records_df)  # Display the number of teams in the DataFrame

records_df.info()


In [ ]:
print(df_turnover_stats.columns)

print(records_df.columns)

In [ ]:
## Merge into the df_turnover_stats dataframe
df_turnover_stats = df_turnover_stats.merge(records_df[['Team', 'Wins', 'Losses', 'Ties']], left_on='team', right_on='Team', how='left')

In [ ]:
### Fuzzy Match the Team Names
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Create a list of all the uniqueteam names in the records_df
record_team_names = list(records_df['Team'].unique())

# Create a list of all the unique team names in the df_turnover_stats
turnover_team_names = list(df_turnover_stats['team'].unique())

# Sort both Lists alphabetically
record_team_names.sort()
turnover_team_names.sort()



## Print the length of each list and the first 5 items in each list
print(len(record_team_names))
print(len(turnover_team_names))
print(record_team_names[:5])
print(turnover_team_names[:5])

# # # Create an empty dataframe to store the fuzzy match results
# fuzzy_match_results = pd.DataFrame(columns=['turnover_team', 'records_Team', 'Score'])

# # Iterate through each team name in turnover_team_names
# for turnover_team in turnover_team_names:
#     # Use fuzzy matching to find the closest match in record_team_names
#     match, score = process.extractOne(turnover_team, record_team_names)
    
#     # Append the result to fuzzy_match_results DataFrame
#     fuzzy_match_results = fuzzy_match_results.append({
#         'turnover_team': turnover_team,
#         'records_Team': match,
#         'Score': score
#     }, ignore_index=True)

# # Display the fuzzy_match_results to see how well the team names matched
# print(fuzzy_match_results)



## Find The number of games that Do Not Have A Line Chart

In [ ]:
# Query to get the total number of unique games in the linescore table
query_total_games = "SELECT COUNT(DISTINCT Game_ID) FROM linescore;"
total_games = conn.execute(query_total_games).fetchone()[0] // 2  # Each game appears twice (one for each team)

# Query to get the total number of unique games in the line_chart table
query_total_games_line_chart = "SELECT COUNT(DISTINCT Game_ID) FROM line_chart;"
total_games_line_chart = conn.execute(query_total_games_line_chart).fetchone()[0]

# Calculate the percentage of games with incomplete line_chart data
percentage_incomplete = ((total_games - total_games_line_chart) / total_games) * 100

total_games, total_games_line_chart, percentage_incomplete



# Face Offs

In [ ]:
# Query to get the face-off related columns from the player_stats table
query_faceoff_data = """
SELECT Team, Player, FOW, FOL, Game_ID 
FROM player_stats
WHERE FOW IS NOT NULL AND FOL IS NOT NULL;
"""

# Execute the query and convert to a DataFrame
faceoff_data_df = pd.read_sql(query_faceoff_data, conn)

# Convert FOW and FOL to numeric types
faceoff_data_df["FOW"] = pd.to_numeric(faceoff_data_df["FOW"], errors='coerce')
faceoff_data_df["FOL"] = pd.to_numeric(faceoff_data_df["FOL"], errors='coerce')

# Calculate the overall face-off win/lose percentage for each team
team_faceoff_summary = faceoff_data_df.groupby("Team").agg({'FOW': 'sum', 'FOL': 'sum'}).reset_index()
team_faceoff_summary["Total_Attempts"] = team_faceoff_summary["FOW"] + team_faceoff_summary["FOL"]
team_faceoff_summary["Team_Win_Percentage"] = (team_faceoff_summary["FOW"] / team_faceoff_summary["Total_Attempts"]) * 100

team_faceoff_summary.head()  # Display the first few rows


In [ ]:

# Execute the query and convert to a DataFrame
faceoff_data_df = pd.read_sql(query_faceoff_data, conn)

# Calculate the total face-off attempts for each player
faceoff_data_df["Total_Attempts"] = faceoff_data_df["FOW"] + faceoff_data_df["FOL"]
faceoff_data_df["Win_Percentage"] = (faceoff_data_df["FOW"] / faceoff_data_df["Total_Attempts"]) * 100

# Initialize an empty DataFrame to store the top 3 players for each team
top_players_50th_df = pd.DataFrame(columns=['Team', 'Player', 'Win_Percentage', 'Total_Attempts'])

# Loop through each team to calculate the 50th percentile threshold and find the top 3 players
for team, team_data in faceoff_data_df.groupby("Team"):
    threshold_50th = team_data["Total_Attempts"].quantile(0.5)
    filtered_players = team_data[team_data["Total_Attempts"] > threshold_50th]
    top_3_players = filtered_players.sort_values("Win_Percentage", ascending=False).head(3)[['Team', 'Player', 'Win_Percentage', 'Total_Attempts']]
    top_players_50th_df = pd.concat([top_players_50th_df, top_3_players])

top_players_50th_df.reset_index(drop=True, inplace=True)

# Reshape to wide format
top_players_wide_df = pd.DataFrame()

for team, team_data in top_players_50th_df.groupby("Team"):
    team_row = pd.DataFrame({"Team": [team]})
    for i, (_, row) in enumerate(team_data.iterrows()):
        team_row[f"Player_{i+1}"] = row["Player"]
        team_row[f"Player_{i+1}_Win_Percentage"] = row["Win_Percentage"]
        team_row[f"Player_{i+1}_Attempts"] = row["Total_Attempts"]
    top_players_wide_df = pd.concat([top_players_wide_df, team_row])

top_players_wide_df.reset_index(drop=True, inplace=True)

In [ ]:
top_players_wide_df.head()  # Display the first few rows

## merge into larger df

In [ ]:
#### Merge the faceoff dfs into the df_turnover_stats dataframe

df_faceoffs = team_faceoff_summary.merge(top_players_wide_df, on='Team', how='left')

# Add 'FO_' to the start of each column name except the 'Team' column make all column names lowercase

df_faceoffs.columns = ['Team'] + ['FO_' + col if col != 'Team' else col for col in df_faceoffs.columns[1:]]
df_faceoffs.columns = [col.lower() for col in df_faceoffs.columns]



df_faceoffs.head()

## Merge Faceoff data into larger dataframe

In [ ]:
# df_turnover_stats.head()

In [ ]:
## Merge the df_faceoffs dataframe into the df_turnover_stats dataframe

df_turnover_stats = df_turnover_stats.merge(df_faceoffs, on='team', how='left')


# Store the Large Dataframe under a new name
df_final = df_turnover_stats

# Save the dataframe as a csv file
df_final.to_csv('../TEMP/Team_Table with_Face_Off.csv', index=False)

print(df_final.shape)
print(df_final.columns)
# df_turnover_stats.head()
# df_turnover_stats.shape
# df_turnover_stats.describe()

# Rename Dataframe and Get to graphical portion

## Create a dataframe with very distint and styled  stats to output an easy to inteperate table or chart


In [ ]:
## Rename the above dataframe
summary_df = df_turnover_stats

# Calculate the ranks for 'age_current' and 'exp_current'  and Draft Score and add them as new columns to the DataFrame
# For Age and Experience, the lower the number, the higher the rank
# For Draft Score, the higher the number, the higher the rank
# Also Store the ranks as integers instead of floats

summary_df['age_rank_current'] = summary_df['age_current'].rank(ascending=False).astype(int)
summary_df['exp_rank_current'] = summary_df['exp_current'].rank(ascending=False).astype(int)
summary_df['draft_score_rank'] = summary_df['Normalized_Draft_Score'].rank(ascending=False).astype(int)
# make one for turnover rate as well - the higher the value the higher the ranking
summary_df['turnover_rate_rank'] = summary_df['turnover_rate'].rank(ascending=False).astype(int)

# Sort the DataFrame again based on 'team' for better visualization
summary_df = summary_df.sort_values(by='team')


keep_cols = ['team',  'Wins', 'Losses',   'Ties',
             'turnover_rate',  'turnover_rate_rank',
             'age_current', 'age_rank_current',
            'exp_current', 'exp_rank_current',
            'Drafted_Count', 'Normalized_Draft_Score']
summary_df = summary_df[keep_cols]


# Display the first few rows of the updated summary DataFrame
summary_df.head()



In [ ]:
summary_df.info()